In [ ]:
%pip install --upgrade pip

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install yolov5

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/953.5 kB ? eta -:--:--
   --------------------------------------- 953.5/953.5 kB 22.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   --------- ------------------------------ 9.4/38.8 MB 58.8 MB/s eta 0:00:01
   ----------- ---------------------------- 11.5/38.8 MB 27.8 MB/s eta 0:00:01
   ------------------------ --------------- 23.3/38.8 MB 37.9 MB/s eta 0:00:01
   ------------------------------------ --- 35.4/38.8 MB 43.3 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 42.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import yaml
from pathlib import Path
from yolov5 import train


# 데이터셋 경로 설정
data_yaml = "data/dataset.yaml"  # 데이터셋의 YAML 설정 파일 경로

# 3. 학습 파라미터 설정
training_args = {
    "data": data_yaml,  # 데이터셋 설정 파일 (YOLO 포맷 필요)
    "weights": "yolov5n.pt",  # 사전 학습된 모델 사용
    "epochs": 50,  # 학습 반복 횟수
    "batch-size": 16,  # 배치 크기
    "img-size": 640,  # 입력 이미지 크기
    "device": 0 if torch.cuda.is_available() else 'cpu',  # GPU 사용 여부
    "project": "custom_yolo",  # 결과 저장 폴더
    "name": "exp",  # 실험명
    "exist-ok": True  # 기존 실험 폴더 덮어쓰기
}

# 4. YOLOv5 모델 학습 실행
train.run(**training_args)

# 5. 학습된 모델 저장 경로
best_model_path = Path("runs/train/exp/weights/best.pt")
if best_model_path.exists():
    print(f"모델 학습 완료! 저장 경로: {best_model_path}")
else:
    print("모델 학습 중 오류 발생!")

# 6. 학습된 모델을 로드하여 사용
model = torch.hub.load('ultralytics/yolov5', 'custom', path=best_model_path, force_reload=True)

# 7. 테스트 이미지 추론 수행
img_path = "test.jpg"  # 테스트할 이미지 파일 경로
results = model(img_path)

# 8. 결과 출력 및 저장
results.show()  # 감지된 객체 표시
results.save()  # 결과 이미지 저장


In [ ]:
%pip install -r https://github.com/ultralytics/yolov5/raw/master/requirements.txt


In [ ]:
%cd C:/Git/ml-experiments/YOLO
!git pull


C:\Git\ml-experiments\YOLO
Already up to date.


In [ ]:
import torch
import traceback
import sys
import argparse
from yolov5 import train
from pathlib import Path

try:
    # 데이터셋 설정 파일 경로 (이미지 폴더 기준)
    data_yaml = "C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml"

    # 올바른 하이퍼파라미터 파일 찾기
    hyp_candidates = [
        "C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml",
        "C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-high.yaml",
        "C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.no-augmentation.yaml"
    ]

    # 사용할 hyp.yaml 파일 찾기
    hyp_path = None
    for hyp_file in hyp_candidates:
        if Path(hyp_file).exists():
            hyp_path = hyp_file
            break

    if not hyp_path:
        raise FileNotFoundError(
            "❌ 사용할 수 있는 하이퍼파라미터 설정 파일(hyp.yaml)이 없습니다. "
            "YOLOv5 최신 버전을 다운로드하거나 'yolov5/data/hyps/' 폴더에 적절한 YAML 파일을 추가하세요."
        )

    # YOLOv5 학습 실행을 위한 argparse.Namespace 생성
    opt = argparse.Namespace(
        data=data_yaml,  # 데이터셋 설정 파일
        weights="C:/Git/ml-experiments/YOLO/yolov5n.pt",  # YOLOv5n 사전 학습된 모델
        epochs=50,  # 학습 반복 횟수
        batch_size=16,  # 배치 크기
        imgsz=640,  # 입력 이미지 크기
        device="0" if torch.cuda.is_available() else "cpu",  # GPU 사용 여부
        project="C:/Git/ml-experiments/YOLO/yolov5/runs/train",  # 결과 저장 폴더
        name="exp",  # 실험명
        exist_ok=True,  # 기존 폴더 덮어쓰기 허용
        resume=False,  # 🚀 추가된 속성 (resume 에러 방지)
        cfg="",  # 🚀 None 대신 빈 문자열로 설정
        hyp=hyp_path,  # 🚀 올바른 hyp.yaml 경로 사용
        single_cls=False,  # 🚀 추가된 속성 (YOLOv5 내부에서 필요)
        evolve=False,  # 🚀 추가된 속성 (YOLOv5 내부에서 필요)
        workers=8,  # 데이터 로딩에 사용할 CPU 스레드 개수
        freeze=0,  # 특정 레이어를 동결할지 여부
        nosave=False,  # 모델 저장 여부
        noval=False  # 검증 과정 포함 여부
    )

    print(f"🚀 모델 학습 시작... (사용할 hyp.yaml: {hyp_path})")

    # YOLOv5 학습 실행
    train.main(opt)

    # 학습된 모델 저장 경로
    best_model_path = Path("C:/Git/ml-experiments/YOLO/yolov5/runs/train/exp/weights/best.pt")

    # 학습 완료 여부 확인
    if best_model_path.exists():
        print(f"✅ 모델 학습 완료! 저장 경로: {best_model_path}")
    else:
        print("❌ 모델 학습 중 오류 발생! (모델이 저장되지 않음)")

except Exception as e:
    print("🔥 오류 발생!")
    print(traceback.format_exc())  # 전체 오류 메시지 출력
    sys.exit(1)  # 시스템 강제 종료


train: data=C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml, weights=C:/Git/ml-experiments/YOLO/yolov5n.pt, epochs=50, batch_size=16, imgsz=640, device=0, project=C:/Git/ml-experiments/YOLO/yolov5/runs/train, name=exp, exist_ok=True, resume=False, cfg=, hyp=C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml, single_cls=False, evolve=False, workers=8, freeze=0, nosave=False, noval=False
YOLOv5  2025-2-12 Python-3.12.2 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0


🚀 모델 학습 시작... (사용할 hyp.yaml: C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml)
requirements: C:\Git\ml-experiments\.venv\Lib\site-packages\requirements.txt not found, check failed.
🔥 오류 발생!
Traceback (most recent call last):
  File "C:\Users\Home\AppData\Local\Temp\ipykernel_22692\3607103148.py", line 57, in <module>
    train.main(opt)
  File "c:\Git\ml-experiments\.venv\Lib\site-packages\yolov5\train.py", line 615, in main
    train(opt.hyp, opt, device, callbacks)
  File "c:\Git\ml-experiments\.venv\Lib\site-packages\yolov5\train.py", line 103, in train
    plots = not evolve and not opt.noplots  # create plots
                               ^^^^^^^^^^^
AttributeError: 'Namespace' object has no attribute 'noplots'



SystemExit: 1

In [ ]:
%cd C:/Git/ml-experiments/YOLO
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -r requirements.txt


In [ ]:
%tb

SystemExit: 1

In [ ]:
import os

labels_path = "C:/Git/ml-experiments/YOLO/yolo_dataset/labels/train"
for label_file in os.listdir(labels_path):
    with open(os.path.join(labels_path, label_file), "r", encoding="ISO-8859-1") as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])  # 클래스 ID 추출
            if class_id >= 41:  # nc보다 큰 값이 있는지 확인
                print(f"🚨 문제 발생! {label_file} 파일에서 {class_id}가 발견됨!")


ValueError: invalid literal for int() with base 10: '\x00\x05\x16\x07\x00\x02\x00\x00Mac'

In [ ]:
import torch
import traceback
import sys
import argparse
from yolov5 import train
from pathlib import Path
import os

try:
    # YOLOv5 실행 폴더로 변경 (필수)
    os.chdir("C:/Git/ml-experiments/YOLO/yolov5")

    # 데이터셋 설정 파일 경로
    data_yaml = "C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml"

    # 하이퍼파라미터 파일 설정
    hyp_path = "C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml"

    # YOLOv5 학습 실행을 위한 argparse.Namespace 생성
    opt = argparse.Namespace(
        # 필수 YOLOv5 학습 파라미터
        data=data_yaml,  # 데이터셋 설정 파일
        weights="C:/Git/ml-experiments/YOLO/yolov5n.pt",  # 사전 학습된 YOLOv5 모델
        epochs=50,  # 학습 반복 횟수
        batch_size=16,  # 배치 크기
        imgsz=640,  # 입력 이미지 크기
        device="0" if torch.cuda.is_available() else "cpu",  # GPU 사용 여부
        project="C:/Git/ml-experiments/YOLO/yolov5/runs/train",  # 결과 저장 폴더
        name="exp",  # 실험명
        exist_ok=True,  # 기존 폴더 덮어쓰기 허용
        resume=False,  # 🚀 학습 재개 여부
        cfg="",  # 🚀 None 대신 빈 문자열로 설정
        hyp=hyp_path,  # 🚀 하이퍼파라미터 파일 지정

        # 추가 YOLOv5 옵션 (모든 필요한 속성 포함)
        single_cls=False,  # 🚀 단일 클래스 여부
        evolve=False,  # 🚀 하이퍼파라미터 진화 사용 여부
        workers=8,  # 🚀 데이터 로딩에 사용할 CPU 스레드 개수
        freeze=0,  # 🚀 특정 레이어를 동결할지 여부
        nosave=False,  # 🚀 학습된 모델 저장 여부
        noval=False,  # 🚀 검증 과정 포함 여부
        noplots=False,  # 🚀 시각화 플롯 저장 여부
        seed=0,  # 🚀 랜덤 시드 고정 (재현성)
        s3_upload_dir=None,  # 🚀 S3 업로드 경로 (기본 None)
        upload_dataset=False,  # 🚀 S3 업로드 여부
        optimizer="SGD",  # 🚀 옵티마이저 설정 (SGD, Adam, AdamW)
        cos_lr=False,  # 🚀 Cosine LR 스케줄링 여부
        sync_bn=False,  # 🚀 Sync BatchNorm 사용 여부
        cache="ram",  # 🚀 데이터 로딩 속도 향상을 위한 캐시 ("ram" 또는 "disk" 또는 False)
        rect=False,  # 🚀 이미지 크기를 일정하게 유지할지 여부
        image_weights=False,  # 🚀 클래스별 데이터 불균형을 보정할지 여부
        quad=False,  # 🚀 4배수 데이터 로딩 사용 여부 (TPU에서 사용)
        save_period=-1,  # 🚀 모델 저장 주기 (-1은 마지막만 저장)
        local_rank=-1,  # 🚀 분산 학습 사용 여부 (-1은 비활성화)
        lr0=0.01,  # 🚀 초기 학습률
        lrf=0.1,  # 🚀 최종 학습률
        momentum=0.937,  # 🚀 모멘텀 설정
        weight_decay=0.0005,  # 🚀 가중치 감소 (Regularization)
        warmup_epochs=3.0,  # 🚀 워밍업 에포크 수
        warmup_momentum=0.8,  # 🚀 워밍업 모멘텀
        warmup_bias_lr=0.1  # 🚀 워밍업 편향 학습률
    )

    print(f"🚀 모델 학습 시작... (사용할 hyp.yaml: {hyp_path})")

    # YOLOv5 학습 실행
    train.main(opt)

    # 학습된 모델 저장 경로
    best_model_path = Path("C:/Git/ml-experiments/YOLO/yolov5/runs/train/exp/weights/best.pt")

    # 학습 완료 여부 확인
    if best_model_path.exists():
        print(f"✅ 모델 학습 완료! 저장 경로: {best_model_path}")
    else:
        print("❌ 모델 학습 중 오류 발생! (모델이 저장되지 않음)")

except Exception as e:
    print("🔥 오류 발생!")
    print(traceback.format_exc())  # 전체 오류 메시지 출력
    sys.exit(1)  # 시스템 강제 종료


train: data=C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml, weights=C:/Git/ml-experiments/YOLO/yolov5n.pt, epochs=50, batch_size=16, imgsz=640, device=0, project=C:/Git/ml-experiments/YOLO/yolov5/runs/train, name=exp, exist_ok=True, resume=False, cfg=, hyp=C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml, single_cls=False, evolve=False, workers=8, freeze=0, nosave=False, noval=False, noplots=False, seed=0, s3_upload_dir=None, upload_dataset=False, optimizer=SGD, cos_lr=False, sync_bn=False, cache=ram, rect=False, image_weights=False, quad=False, save_period=-1, local_rank=-1, lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1
YOLOv5  2025-2-12 Python-3.12.2 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, i

🚀 모델 학습 시작... (사용할 hyp.yaml: C:/Git/ml-experiments/YOLO/yolov5/data/hyps/hyp.scratch-low.yaml)
requirements: C:\Git\ml-experiments\.venv\Lib\site-packages\requirements.txt not found, check failed.


Overriding model.yaml nc=80 with nc=41

                 from  n    params  module                                  arguments                     
  0                -1  1      1760  yolov5.models.common.Conv               [3, 16, 6, 2, 2]              
  1                -1  1      4672  yolov5.models.common.Conv               [16, 32, 3, 2]                
  2                -1  1      4800  yolov5.models.common.C3                 [32, 32, 1]                   
  3                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  4                -1  2     29184  yolov5.models.common.C3                 [64, 64, 2]                   
  5                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  6                -1  3    156928  yolov5.models.common.C3                 [128, 128, 3]                 
  7                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]            

🔥 오류 발생!
Traceback (most recent call last):
  File "C:\Users\Home\AppData\Local\Temp\ipykernel_1716\1782156597.py", line 67, in <module>
    train.main(opt)
  File "c:\Git\ml-experiments\.venv\Lib\site-packages\yolov5\train.py", line 615, in main
    train(opt.hyp, opt, device, callbacks)
  File "c:\Git\ml-experiments\.venv\Lib\site-packages\yolov5\train.py", line 226, in train
    assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data}. Possible class labels are 0-{nc - 1}'
AssertionError: Label class 125 exceeds nc=41 in C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml. Possible class labels are 0-40



SystemExit: 1

In [ ]:
from yolov5.utils.general import check_dataset
data_yaml = "C:/Git/ml-experiments/YOLO/yolo_dataset/dataset.yaml"

try:
    dataset_info = check_dataset(data_yaml)
    print("✅ YOLOv5가 dataset.yaml을 정상적으로 찾았습니다.")
    print(dataset_info)
except Exception as e:
    print("❌ YOLOv5에서 dataset.yaml을 찾을 수 없습니다.")
    print(e)


100%|██████████| 22.2M/22.2M [00:01<00:00, 21.7MB/s]

✅ YOLOv5가 dataset.yaml을 정상적으로 찾았습니다.
{'train': 'C:\\Git\\ml-experiments\\YOLO\\yolo_dataset\\images\\train', 'val': 'C:\\Git\\ml-experiments\\YOLO\\yolo_dataset\\images\\val', 'nc': 41, 'names': {0: '占신쏙옙', 1: '占쏙옙화占쏙옙', 2: '체占쏙옙', 3: '占쏙옙占쏙옙占쏙옙', 4: '占쏙옙占쏙옙占쏙옙', 5: '천占쏙옙占쏙옙占쏙옙占쏙옙', 6: '占쏙옙占쏙옙占쏙옙占쏙옙', 7: '占쏙옙占썹베占쏙옙', 8: '占쏙옙占쏙옙占', 9: '占쌘듸옙', 10: '占쏙옙占쏙옙(占쏙옙占쏙옙)', 11: '占쏙옙', 12: '占쏙옙占쏙옙', 13: '占쌈뤄옙(占쏙옙占쏙옙)', 14: '占쏙옙占', 15: '占쏙옙', 16: '占쏙옙占', 17: '占쏙옙占쏙옙', 18: '占쏙옙占쏙옙', 19: '占쏙옙', 20: '천占쏙옙占쏙옙', 21: '占싼띰옙占', 22: '占쏙옙占쏙옙', 23: '占쏙옙占쏙옙', 24: '占쏙옙占쏙옙', 25: '占쏙옙占', 26: '占쏙옙占싸억옙占쏙옙', 27: '占쌕놂옙占쏙옙', 28: '占쏙옙占', 29: '占싣븝옙카占쏙옙', 30: '占쌘몌옙', 31: '占싸몌옙占쏙옙', 32: '占쏙옙占쏙옙占쏙옙틴', 33: '占쏙옙占쏙옙', 34: '占쏙옙占싣뱄옙', 35: '占쏙옙치', 36: '占쏙옙占식억옙', 37: '占쏙옙占쏙옙탄', 38: '占싻쇽옙占식몌옙占쏙옙', 39: '占쏙옙占쏙옙', 40: '키占쏙옙'}, 'path': WindowsPath('C:/Git/ml-experiments/.venv/Lib/site-packages/yolov5')}
